# U.S. Congress

In [23]:
import polars as pl
import altair as alt
from datetime import date

alt.theme.enable('latimes')

file = 'https://unitedstates.github.io/congress-legislators/legislators-current.csv'

df = pl.read_csv(file, schema_overrides={'birthday': pl.Date})

df = df.with_columns(
    ((date.today() - pl.col('birthday')).dt.total_days() / 365.25).alias('age'),
)

df[:4]

last_name,first_name,middle_name,suffix,nickname,full_name,birthday,gender,type,state,district,senate_class,party,url,address,phone,contact_form,rss_url,twitter,twitter_id,facebook,youtube,youtube_id,mastodon,bioguide_id,thomas_id,opensecrets_id,lis_id,fec_ids,cspan_id,govtrack_id,votesmart_id,ballotpedia_id,washington_post_id,icpsr_id,wikipedia_id,age
str,str,str,str,str,str,date,str,str,str,i64,i64,str,str,str,str,str,str,str,i64,str,str,str,str,str,i64,str,str,str,i64,i64,i64,str,str,i64,str,f64
"""Cantwell""","""Maria""",null,null,null,"""Maria Cantwell""",1958-10-13,"""F""","""sen""","""WA""",null,1,"""Democrat""","""https://www.cantwell.senate.go…","""511 Hart Senate Office Buildin…","""202-224-3441""","""https://www.cantwell.senate.go…","""http://www.cantwell.senate.gov…","""SenatorCantwell""",117501995,"""senatorcantwell""","""SenatorCantwell""","""UCN52UDqKgvHRk39ncySrIMw""",null,"""C000127""",172,"""N00007836""","""S275""","""S8WA00194,H2WA01054""",26137,300018,27122,"""Maria Cantwell""",null,39310,"""Maria Cantwell""",66.992471
"""Klobuchar""","""Amy""","""Jean""",null,null,"""Amy Klobuchar""",1960-05-25,"""F""","""sen""","""MN""",null,1,"""Democrat""","""https://www.klobuchar.senate.g…","""425 Dirksen Senate Office Buil…","""202-224-3244""","""https://www.klobuchar.senate.g…",null,"""SenAmyKlobuchar""",22044727,null,"""senatorklobuchar""","""UCvdeJsDsV51tFb_hVqvtYGA""",null,"""K000367""",1826,"""N00027500""","""S311""","""S6MN00267""",83701,412242,65092,"""Amy Klobuchar""",null,40700,"""Amy Klobuchar""",65.377139
"""Sanders""","""Bernard""",null,null,"""Bernie""","""Bernard Sanders""",1941-09-08,"""M""","""sen""","""VT""",null,1,"""Independent""","""https://www.sanders.senate.gov""","""332 Dirksen Senate Office Buil…","""202-224-5141""","""https://www.sanders.senate.gov…","""http://www.sanders.senate.gov/…","""SenSanders""",29442313,"""senatorsanders""","""senatorsanders""","""UCD_DaKNac0Ta-2PeHuoQ1uA""",null,"""S000033""",1010,"""N00000528""","""S313""","""H8VT01016,S4VT00033""",994,400357,27110,"""Bernie Sanders""",null,29147,"""Bernie Sanders""",84.087611
"""Whitehouse""","""Sheldon""",null,null,null,"""Sheldon Whitehouse""",1955-10-20,"""M""","""sen""","""RI""",null,1,"""Democrat""","""https://www.whitehouse.senate.…","""530 Hart Senate Office Buildin…","""202-224-2921""","""https://www.whitehouse.senate.…","""http://www.whitehouse.senate.g…","""SenWhitehouse""",242555999,"""SenatorWhitehouse""","""SenatorWhitehouse""","""UCnG0N70SNBkNqvIMLodPTIA""",null,"""W000802""",1823,"""N00027533""","""S316""","""S6RI00221""",92235,412247,2572,"""Sheldon Whitehouse""",null,40704,"""Sheldon Whitehouse""",69.97399


## Gender Breakdown

In [40]:
df_party_gender_agg = (
    df
    .select(
        pl.col('party'),
        pl.col('gender'),
        pl.len().over(['party', 'gender']).alias('count'),
        (pl.len().over(['party', 'gender']) / pl.len().over('party')).alias('percentage'),
    )
    .unique()
)

display(df_party_gender_agg)
chart = alt.Chart(
    df_party_gender_agg,
    title="U.S. Congress",
).mark_bar(
    stroke='black',
    opacity=0.8,
).encode(
    x=alt.X('gender', title=''),
    y=alt.Y('count', title='no. of members'),
    column=alt.Column('party', title=''),
    color=alt.Color('gender', scale=alt.Scale(scheme='set3')),
).properties(
    width=250,
    height=400,
)

percentage_chart = alt.Chart(
    df_party_gender_agg,
    title="U.S. Congress",
).mark_bar(
    stroke='black',
    opacity=0.8,
).encode(
    x=alt.X('gender', title=''),
    y=alt.Y('percentage', title='% of members').axis(format='%'),
    column=alt.Column('party', title=''),
    color=alt.Color('gender', scale=alt.Scale(scheme='set3')),
).properties(
    width=250,
    height=400,
)

chart.show()
percentage_chart.show()

party,gender,count,percentage
str,str,u32,f64
"""Republican""","""F""",43,0.156364
"""Independent""","""M""",2,1.0
"""Democrat""","""F""",112,0.429119
"""Democrat""","""M""",149,0.570881
"""Republican""","""M""",232,0.843636


alt.Chart(...)

alt.Chart(...)

## Age

In [20]:
display(
    df.select(
        pl.col('full_name'),
        pl.col('gender'),
        pl.col('birthday'),
        pl.col('state'),
        pl.col('party'),
        pl.col('age').round(2),
    )
    .sort('age', descending=True)
    .head(9)
)

display(
    df.select(
        pl.col('full_name'),
        pl.col('gender'),
        pl.col('birthday'),
        pl.col('state'),
        pl.col('party'),
        pl.col('age').round(2),
    )
    .sort('age', descending=True)
    .tail(9)
)

display(
    df.group_by('party')
    .agg(
        pl.col('age').min().round(2).alias('youngest'),
        pl.col('age').max().round(2).alias('oldest'),
        pl.col('age').mean().round(2).alias('average_age'),
    )
)

full_name,gender,birthday,state,party,age
str,str,date,str,str,f64
"""Chuck Grassley""","""M""",1933-09-17,"""IA""","""Republican""",92.06
"""Eleanor Holmes Norton""","""F""",1937-06-13,"""DC""","""Democrat""",88.33
"""Harold Rogers""","""M""",1937-12-31,"""KY""","""Republican""",87.78
"""Maxine Waters""","""F""",1938-08-15,"""CA""","""Democrat""",87.15
"""Steny H. Hoyer""","""M""",1939-06-14,"""MD""","""Democrat""",86.32
"""Nancy Pelosi""","""F""",1940-03-26,"""CA""","""Democrat""",85.54
"""James E. Clyburn""","""M""",1940-07-21,"""SC""","""Democrat""",85.22
"""Bernard Sanders""","""M""",1941-09-08,"""VT""","""Independent""",84.09
"""Danny K. Davis""","""M""",1941-09-06,"""IL""","""Democrat""",84.09


full_name,gender,birthday,state,party,age
str,str,date,str,str,f64
"""Alexandria Ocasio-Cortez""","""F""",1989-10-13,"""NY""","""Democrat""",35.99
"""Robert P. Bresnahan, Jr.""","""M""",1990-04-22,"""PA""","""Republican""",35.47
"""Sarah McBride""","""F""",1990-08-09,"""DE""","""Democrat""",35.17
"""Pablo José Hernández""","""M""",1991-05-11,"""PR""","""Democrat""",34.42
"""Abraham J. Hamadeh""","""M""",1991-05-15,"""AZ""","""Republican""",34.41
"""Yassamin Ansari""","""F""",1992-04-07,"""AZ""","""Democrat""",33.51
"""Addison P. McDowell""","""M""",1994-01-21,"""NC""","""Republican""",31.72
"""Brandon Gill""","""M""",1994-02-26,"""TX""","""Republican""",31.62
"""Maxwell Frost""","""M""",1997-01-17,"""FL""","""Democrat""",28.73


party,youngest,oldest,average_age
str,f64,f64,f64
"""Democrat""",28.73,88.33,59.74
"""Republican""",31.62,92.06,59.04
"""Independent""",81.53,84.09,82.81


In [35]:
alt.Chart(
    df.filter(pl.col('party').is_in(['Democrat', 'Republican'])),
    title='U.S. Congress — Age Distribution',
).mark_bar(
    stroke='black',
    opacity=0.85,
).encode(
    x=alt.X('age', bin=True, title='age'),
    y=alt.Y('count()', title='members'),
    row=alt.Row('party', title=''),
    color=alt.Color('party', title='', scale=alt.Scale(domain=['Democrat', 'Republican'], range=['#0080ff', '#f15f49'])),
).properties(
    width=600,
    height=350,
)

alt.Chart(...)